# PMU Disturbance Analysis - Data Loading & Quality Assessment

This notebook:
1. Loads PMU and disturbance data from Excel
2. Performs comprehensive data quality assessment
3. Validates data integrity and linkages
4. Saves cleaned data for subsequent analyses

**Input**: `data/PMU_disturbance.xlsx` (2 sheets: PMUs, Disturbances)

**Output**: `outputs/data/cleaned_data.parquet`, `outputs/reports/data_quality_report.txt`

In [ ]:
# Import libraries
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Import project modules
from src.data_loader import load_all_data, get_data_summary
import config

# Set visualization style
sns.set_style(config.PLOT_SETTINGS['style'])
plt.rcParams['figure.figsize'] = config.DEFAULT_FIGSIZE
plt.rcParams['font.size'] = config.PLOT_SETTINGS['font_size']

print("Libraries loaded successfully!")

## 1. Load Data

In [ ]:
# Load all datasets
pmu_df, disturbance_df, merged_df = load_all_data(
    config.EXCEL_FILE,
    config.PMU_SHEET,
    config.DISTURBANCE_SHEET
)

print("\nData loaded successfully!")
print(f"PMU data shape: {pmu_df.shape}")
print(f"Disturbance data shape: {disturbance_df.shape}")
print(f"Merged data shape: {merged_df.shape}")

In [ ]:
# Display first few rows of PMU data
print("PMU Data Sample:")
display(pmu_df.head())
print("\nPMU Columns:", list(pmu_df.columns))

In [ ]:
# Display first few rows of disturbance data
print("Disturbance Data Sample:")
display(disturbance_df.head())
print("\nDisturbance Columns:", list(disturbance_df.columns))

## 2. Data Quality Assessment

### 2.1 Missing Values Analysis

In [ ]:
# Missing values in PMU data
pmu_missing = pd.DataFrame({
    'Count': pmu_df.isna().sum(),
    'Percentage': (pmu_df.isna().sum() / len(pmu_df) * 100).round(2)
}).sort_values('Count', ascending=False)

print("Missing Values in PMU Data:")
display(pmu_missing[pmu_missing['Count'] > 0])

In [ ]:
# Missing values in disturbance data
dist_missing = pd.DataFrame({
    'Count': disturbance_df.isna().sum(),
    'Percentage': (disturbance_df.isna().sum() / len(disturbance_df) * 100).round(2)
}).sort_values('Count', ascending=False)

print("Missing Values in Disturbance Data:")
display(dist_missing[dist_missing['Count'] > 0])

In [ ]:
# Visualize missing values
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# PMU missing values
pmu_missing_plot = pmu_missing[pmu_missing['Count'] > 0]
if len(pmu_missing_plot) > 0:
    axes[0].barh(pmu_missing_plot.index, pmu_missing_plot['Percentage'])
    axes[0].set_xlabel('Missing Percentage (%)')
    axes[0].set_title('Missing Values in PMU Data')
    axes[0].grid(axis='x', alpha=0.3)

# Disturbance missing values
dist_missing_plot = dist_missing[dist_missing['Count'] > 0]
if len(dist_missing_plot) > 0:
    axes[1].barh(dist_missing_plot.index, dist_missing_plot['Percentage'])
    axes[1].set_xlabel('Missing Percentage (%)')
    axes[1].set_title('Missing Values in Disturbance Data')
    axes[1].grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.savefig(f"{config.FIGURE_DIR}/static/01_missing_values.png", dpi=config.FIGURE_DPI, bbox_inches='tight')
plt.show()

### 2.2 Duplicate Detection

In [ ]:
# Check for duplicates
pmu_duplicates = pmu_df.duplicated().sum()
dist_duplicates = disturbance_df.duplicated().sum()

print(f"PMU Data Duplicates: {pmu_duplicates}")
print(f"Disturbance Data Duplicates: {dist_duplicates}")

if 'SectionID' in pmu_df.columns:
    pmu_id_duplicates = pmu_df['SectionID'].duplicated().sum()
    print(f"Duplicate SectionIDs in PMU data: {pmu_id_duplicates}")

### 2.3 Data Type Validation

In [ ]:
# Check data types
print("PMU Data Types:")
print(pmu_df.dtypes)
print("\nDisturbance Data Types:")
print(disturbance_df.dtypes)

### 2.4 Outlier Detection (IQR Method)

In [ ]:
# Identify numeric columns for outlier detection
numeric_cols_dist = disturbance_df.select_dtypes(include=[np.number]).columns.tolist()

print(f"Numeric columns in disturbance data: {numeric_cols_dist}")

# Calculate outliers using IQR method
outlier_summary = {}
for col in numeric_cols_dist:
    Q1 = disturbance_df[col].quantile(0.25)
    Q3 = disturbance_df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = ((disturbance_df[col] < lower_bound) | (disturbance_df[col] > upper_bound)).sum()
    outlier_summary[col] = {
        'count': outliers,
        'percentage': (outliers / len(disturbance_df) * 100).round(2)
    }

outlier_df = pd.DataFrame(outlier_summary).T
outlier_df = outlier_df[outlier_df['count'] > 0].sort_values('count', ascending=False)
print("\nOutlier Summary (IQR method):")
display(outlier_df)

### 2.5 Temporal Coverage Check

In [ ]:
# Find datetime columns
datetime_cols = disturbance_df.select_dtypes(include=['datetime64']).columns.tolist()

if datetime_cols:
    print("Temporal Coverage:")
    for col in datetime_cols:
        print(f"\n{col}:")
        print(f"  Start: {disturbance_df[col].min()}")
        print(f"  End: {disturbance_df[col].max()}")
        print(f"  Span: {(disturbance_df[col].max() - disturbance_df[col].min()).days} days")
        print(f"  Missing: {disturbance_df[col].isna().sum()} records")

### 2.6 SectionID Linkage Validation

In [ ]:
# Validate SectionID linkage
if 'SectionID' in pmu_df.columns and 'SectionID' in disturbance_df.columns:
    pmu_sections = set(pmu_df['SectionID'].dropna())
    dist_sections = set(disturbance_df['SectionID'].dropna())
    
    print(f"Unique SectionIDs in PMU data: {len(pmu_sections)}")
    print(f"Unique SectionIDs in Disturbance data: {len(dist_sections)}")
    
    # Sections in disturbances but not in PMU data
    unmatched_sections = dist_sections - pmu_sections
    print(f"\nSections in disturbances without PMU data: {len(unmatched_sections)}")
    if len(unmatched_sections) > 0 and len(unmatched_sections) <= 20:
        print(f"Unmatched sections: {sorted(unmatched_sections)}")
    
    # Sections in PMU data without disturbances
    unused_sections = pmu_sections - dist_sections
    print(f"\nPMU sections without any disturbances: {len(unused_sections)}")
    if len(unused_sections) > 0 and len(unused_sections) <= 20:
        print(f"Unused sections: {sorted(unused_sections)}")

## 3. Basic Descriptive Statistics

In [ ]:
# PMU descriptive statistics
print("PMU Data - Descriptive Statistics:")
display(pmu_df.describe())

In [ ]:
# Disturbance descriptive statistics
print("Disturbance Data - Descriptive Statistics:")
display(disturbance_df.describe())

## 4. Generate Data Quality Report

In [ ]:
# Generate comprehensive data quality report
report_lines = []
report_lines.append("=" * 80)
report_lines.append("PMU DISTURBANCE DATA QUALITY REPORT")
report_lines.append("=" * 80)
report_lines.append("")

# Dataset overview
report_lines.append("1. DATASET OVERVIEW")
report_lines.append("-" * 80)
report_lines.append(f"PMU Records: {len(pmu_df):,}")
report_lines.append(f"Disturbance Records: {len(disturbance_df):,}")
report_lines.append(f"Merged Records: {len(merged_df):,}")
report_lines.append("")

# Missing values
report_lines.append("2. MISSING VALUES")
report_lines.append("-" * 80)
report_lines.append("PMU Data:")
pmu_missing_report = pmu_missing[pmu_missing['Count'] > 0]
if len(pmu_missing_report) == 0:
    report_lines.append("  No missing values detected")
else:
    for col, row in pmu_missing_report.iterrows():
        report_lines.append(f"  {col}: {int(row['Count'])} ({row['Percentage']}%)")
        
report_lines.append("\nDisturbance Data:")
dist_missing_report = dist_missing[dist_missing['Count'] > 0]
if len(dist_missing_report) == 0:
    report_lines.append("  No missing values detected")
else:
    for col, row in dist_missing_report.iterrows():
        report_lines.append(f"  {col}: {int(row['Count'])} ({row['Percentage']}%)")
report_lines.append("")

# Duplicates
report_lines.append("3. DUPLICATE RECORDS")
report_lines.append("-" * 80)
report_lines.append(f"PMU Data: {pmu_duplicates} duplicate rows")
report_lines.append(f"Disturbance Data: {dist_duplicates} duplicate rows")
report_lines.append("")

# Data quality summary
report_lines.append("4. DATA QUALITY SUMMARY")
report_lines.append("-" * 80)
report_lines.append(f"Overall data completeness: {((1 - merged_df.isna().sum().sum() / (len(merged_df) * len(merged_df.columns))) * 100):.2f}%")
report_lines.append("")

# Write report
report_path = Path(config.REPORT_DIR) / 'data_quality_report.txt'
report_path.parent.mkdir(parents=True, exist_ok=True)
with open(report_path, 'w') as f:
    f.write('\n'.join(report_lines))

print("Data quality report generated!")
print(f"Saved to: {report_path}")
print("\n" + "\n".join(report_lines))

## 5. Save Cleaned Data

In [ ]:
# Save cleaned datasets
output_path = Path(config.CLEANED_DATA)
output_path.parent.mkdir(parents=True, exist_ok=True)

# Save merged data as parquet for efficient storage
merged_df.to_parquet(config.CLEANED_DATA, index=False)
print(f"Cleaned data saved to: {config.CLEANED_DATA}")

# Also save individual datasets as CSV for reference
pmu_df.to_csv(Path(config.OUTPUT_DIR) / 'data' / 'pmu_data.csv', index=False)
disturbance_df.to_csv(Path(config.OUTPUT_DIR) / 'data' / 'disturbance_data.csv', index=False)
print("Individual datasets saved as CSV")

print("\nData loading and quality assessment complete!")

## Summary

This notebook has:
- ✅ Loaded PMU and disturbance data from Excel
- ✅ Performed comprehensive data quality assessment
- ✅ Validated SectionID linkages
- ✅ Generated data quality report
- ✅ Saved cleaned data for subsequent analyses

**Next Steps**: Proceed to temporal analysis (Notebook 02)